In [1]:
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate, train_test_split

In [12]:
svd_df = pd.read_csv('animelists_svd.csv')
reader = Reader(rating_scale=(1, 10))
svd_data = Dataset.load_from_df(svd_df, reader)
#svd_train_data, testset = train_test_split(svd_data, test_size=.20)
svd_data = svd_data.build_full_trainset()
svd_test_data = svd_data.build_testset()
algo = SVD(n_factors=5)
algo.fit(svd_data)
predictions = algo.test(svd_test_data)

In [13]:
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm

def get_svd_top_n(predictions, n=10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in tqdm(predictions):
        top_n[uid].append((iid, est, true_r))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:5]
        
    return top_n

top_n = get_svd_top_n(predictions, n=10)

anime_id=defaultdict(list)

for uid, user_ratings in top_n.items():
    anime_id[uid].append([iid for (iid,_,_) in user_ratings])